In [1]:
import spacy
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from bs4 import BeautifulSoup as BS
import glob
import time
from tqdm import tqdm
import re
from nltk import sent_tokenize

(1) On October 7, 2020, we announced that our board of directors authorized and approved a share repurchase program of up to $200.0 million of our outstanding stock. We began repurchasing shares under this program during the first quarter of fiscal year 2021. As of July 31, 2021, we had approximately $37.5 million remaining for future share repurchases under the share repurchase program. The stock repurchase program does not obligate us to repurchase any dollar amount or number of shares, and the program may be suspended or discontinued at any time. There is no stated expiration for the program.

In [2]:
#trying on one file
with open('./data/gwre-20210731.htm', encoding='utf_8') as fi:
        fi = fi.read()
        soup = BS(fi, 'html.parser')

In [3]:
gwre_file = soup.text

In [4]:
sentences_tokenized=sent_tokenize(gwre_file)
sentences_tokenized  

['\ngwre-2021073100015283962021FYfalseP3YP3Mus-gaap:OtherLiabilitiesCurrentus-gaap:OtherLiabilitiesCurrentus-gaap:OtherLiabilitiesCurrent00015283962020-08-012021-07-31iso4217:USD00015283962021-01-31xbrli:shares00015283962021-08-3100015283962021-07-3100015283962020-07-31iso4217:USDxbrli:shares0001528396gwre:SubscriptionAndSupportMember2020-08-012021-07-310001528396gwre:SubscriptionAndSupportMember2019-08-012020-07-310001528396gwre:SubscriptionAndSupportMember2018-08-012019-07-310001528396us-gaap:LicenseMember2020-08-012021-07-310001528396us-gaap:LicenseMember2019-08-012020-07-310001528396us-gaap:LicenseMember2018-08-012019-07-310001528396us-gaap:ServiceMember2020-08-012021-07-310001528396us-gaap:ServiceMember2019-08-012020-07-310001528396us-gaap:ServiceMember2018-08-012019-07-3100015283962019-08-012020-07-3100015283962018-08-012019-07-310001528396us-gaap:CommonStockMemberus-gaap:CommonStockMember2018-07-310001528396us-gaap:AdditionalPaidInCapitalMember2018-07-310001528396us-gaap:Accumul

In [5]:
sentences=[]

for sentence in sentences_tokenized:
    match=re.search(r"(share|stock) repurchase (program)?", sentence)
    if match: 
        match=re.search(r"authorized|approved", sentence)
        if match:
            sentences.append(sentence)  

In [6]:
sentences

['The Court of Chancery of the State of Delaware and the United States District Court for the Northern District of California may also reach different judgments or results than would other courts, including courts where a stockholder considering an action may be located or would otherwise choose to bring the action, and such judgments may be more or less favorable to us than our stockholders.We cannot guarantee that our share repurchase program will be fully consummated or it will enhance stockholder value, and share repurchases could affect the price of our common stock.In October 2020, our board of directors authorized and approved a share repurchase program of up to $200 million of our outstanding common stock, $37.5 million of which remains available for future repurchases as of July\xa031, 2021.',
 'Any future determination as to the declaration and payment of dividends, if any, will be at the discretion of our board of directors and will depend on then existing conditions, includ

In [7]:
paragraph = [re.sub(r"\\[a-z]+\d?", " ", repr(sentence).strip("'")) for sentence in sentences]
paragraph

['The Court of Chancery of the State of Delaware and the United States District Court for the Northern District of California may also reach different judgments or results than would other courts, including courts where a stockholder considering an action may be located or would otherwise choose to bring the action, and such judgments may be more or less favorable to us than our stockholders.We cannot guarantee that our share repurchase program will be fully consummated or it will enhance stockholder value, and share repurchases could affect the price of our common stock.In October 2020, our board of directors authorized and approved a share repurchase program of up to $200 million of our outstanding common stock, $37.5 million of which remains available for future repurchases as of July 31, 2021.',
 'Any future determination as to the declaration and payment of dividends, if any, will be at the discretion of our board of directors and will depend on then existing conditions, including

In [8]:
import spacy
nlp=spacy.load('en_core_web_sm')

nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [9]:
doc=nlp(str(paragraph))
for ent in doc.ents:
  print(ent.text,ent.label_)

'The Court of Chancery of the State of Delaware WORK_OF_ART
the United States District Court GPE
the Northern District LOC
California GPE
October 2020 DATE
up to $200 million MONEY
$37.5 million MONEY
July 31, 2021 DATE
IssuerThe ORG
the fiscal quarter ended July 31, 2021: DATE
millions CARDINAL
May 31 DATE
2021126,688$97.081,250,029$63.9June 1, 2021 – June 30, 2021123,950$109.151,373,979$50.4July 1, 2021 – July 31 DATE
2021115,012$112.281,488,991$37.5(1 CARDINAL
October 7, 2020 DATE
up to $200.0 million MONEY
the United States GPE
Repurchase ProgramIn PERSON
October 2020 DATE
up to $200.0 million MONEY
this Annual Report EVENT
thousands CARDINAL
Fiscal years ended July 31 DATE
113,066 MONEY
5,801)Net CARDINAL
$1.5 million MONEY
fiscal year 2021 DATE
fiscal year 2020 DATE
October 2020 DATE
Company\ ORG
up to $200.0 million MONEY
Company\ ORG


In [10]:
for p in paragraph:
    doc=nlp(p)
    for ent in doc.ents:
        if ent.label_ in ['DATE', 'CARDINAL', 'MONEY']:
            print(ent.text, ent.label_)

October 2020 DATE
up to $200 million MONEY
$37.5 million MONEY
July 31, 2021 DATE
the fiscal quarter ended July 31, 2021: DATE
millions CARDINAL
May 31 DATE
2021126,688$97.081,250,029$63.9June 1, 2021 – June 30, 2021123,950$109.151,373,979$50.4July 1, 2021 – July 31 DATE
2021115,012$112.281,488,991$37.5(1 CARDINAL
October 7, 2020 DATE
up to $200.0 million MONEY
October 2020 DATE
up to $200.0 million MONEY
thousands CARDINAL
Fiscal years ended July 31 DATE
113,066 MONEY
5,801)Net CARDINAL
$1.5 million MONEY
fiscal year 2021 DATE
fiscal year 2020 DATE
October 2020 DATE
up to $200.0 million MONEY


Next step is to write functions, iteratate through each file one by one instead of glob. Maybe use next(). First function to import data,clean,tokenize. Second function with regex. Third function to NER

In [27]:
def import_10K(filepath):
    with open(filepath, encoding = 'utf-8') as fi:
        soup = BS(fi, 'html.parser')
        ten_k = soup.text
        sentences_tokenized=sent_tokenize(ten_k)
        
    return sentences_tokenized

In [33]:
for filepath in ('./data/*'):
    import_10K(filepath)

PermissionError: [Errno 13] Permission denied: '.'

In [15]:
tenkdict = {}

for filepath in tqdm('./data/*'):
    title = filepath
    ten_k = import_10K(filepath)
    
    tenkdict[title] = ten_k

  0%|          | 0/8 [00:00<?, ?it/s]


PermissionError: [Errno 13] Permission denied: '.'